In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 8, 3, padding=1,bias=False) #input - 28 ? Output - 28 ? RF - 3
        self.dropout1 = nn.Dropout(0.1)
        self.bnorm1 = nn.BatchNorm2d(8)

        self.conv2 = nn.Conv2d(8, 8, 3, padding=1,bias=False) #input - 28 ? Output - 28 ? RF - 5
        self.dropout2 = nn.Dropout(0.1)
        self.bnorm2 = nn.BatchNorm2d(8)

        self.pool1 = nn.MaxPool2d(2, 2) #input - 28 ? Output - 14 ? RF - 10

        self.conv3 = nn.Conv2d(8, 8, 3, padding=1,bias=False) #input - 14 ? Output - 14 ? RF - 12
        self.dropout3 = nn.Dropout(0.1)
        self.bnorm3 = nn.BatchNorm2d(8)
        
        self.conv4 = nn.Conv2d(8, 16, 3, padding=1,bias=False) #input - 14 ? Output - 14 ? RF - 14
        self.dropout4 = nn.Dropout(0.1)
        self.bnorm4 = nn.BatchNorm2d(16)

        self.pool2 = nn.MaxPool2d(2, 2) #input - 14 ? Output - 7 ? RF - 28

        self.conv5 = nn.Conv2d(16, 16, 3,bias=False) #input - 7 ? Output - 5 ? RF - 30
        self.dropout5 = nn.Dropout(0.1)
        self.bnorm5 = nn.BatchNorm2d(16)

        self.conv6 = nn.Conv2d(16, 32, 3,bias=False) #input - 5 ? Output - 3 ? RF - 32
        self.dropout6 = nn.Dropout(0.1)
        self.bnorm6 = nn.BatchNorm2d(32)

        self.conv7 = nn.Conv2d(32, 10, 3,bias=False) #input - 3 ? Output - 1 ? RF - 34

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.bnorm1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.bnorm2(x)

        x = self.pool1(x)
       
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout3(x)
        x = self.bnorm3(x)

        x = self.conv4(x)
        x = F.relu(x)
        x = self.dropout4(x)
        x = self.bnorm4(x)
        
        x = self.pool2(x)

        x = self.conv5(x)
        x = F.relu(x)
        x = self.dropout5(x)
        x = self.bnorm5(x)
        
        x = self.conv6(x)
        x = F.relu(x)
        x = self.dropout6(x)
        x = self.bnorm6(x)
        
        x = self.conv7(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

In [12]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
           Dropout-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4            [-1, 8, 28, 28]             576
           Dropout-5            [-1, 8, 28, 28]               0
       BatchNorm2d-6            [-1, 8, 28, 28]              16
         MaxPool2d-7            [-1, 8, 14, 14]               0
            Conv2d-8            [-1, 8, 14, 14]             576
           Dropout-9            [-1, 8, 14, 14]               0
      BatchNorm2d-10            [-1, 8, 14, 14]              16
           Conv2d-11           [-1, 16, 14, 14]           1,152
          Dropout-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [13]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [14]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch = {epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [16]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.02, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0594, Accuracy: 9814/10000 (98.14%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0397, Accuracy: 9878/10000 (98.78%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0202, Accuracy: 9937/10000 (99.37%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0225, Accuracy: 9927/10000 (99.27%)



epoch = 19 loss=0.0011225842172279954 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.34it/s]



Test set: Average loss: 0.0220, Accuracy: 9941/10000 (99.41%)

